In [270]:
from bs4 import BeautifulSoup
import requests
import datetime
from datetime import date 
from datetime import datetime
import pandas as pd 
import re 
import numpy as np
import re
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import sys
import warnings
from datetime import timedelta
import time as time 

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
#Get df of leagues 

def get_pinn_leagues(URL):
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(
        executable_path=chrome_path, options=chrome_options)
    
    browser.get(URL)
    time.sleep(20)

    htmlSource = browser.page_source
    
    soup= BeautifulSoup(htmlSource, 'lxml')
    league_name= []
    links= []
    for element in soup.find_all('a',{'class':'style_noLeftIcon__3p7DJ style_compact__1hNoP style_supportFavoritesList__20Xov'}):
        links.append('https://www.pinnacle.com'+element['href'])
        league_name.append(element.text)
    
    link_frame= pd.DataFrame({
        'league': league_name,
        'link': links})
    
    p= re.compile('.*[^0-9]')
        
    for i in np.arange(0,len(link_frame)):
        link_frame['league'][i]= p.search(link_frame['league'][i]).group()
        
    return(link_frame)


#def 

In [80]:
get_pinn_leagues('https://www.pinnacle.com/en/soccer/leagues')

,league,link
0,England - Premier League,https://www.pinnacle.com/en/soccer/england-pre...
1,Germany - Bundesliga,https://www.pinnacle.com/en/soccer/germany-bun...
2,Italy - Serie A,https://www.pinnacle.com/en/soccer/italy-serie...
3,Spain - La Liga,https://www.pinnacle.com/en/soccer/spain-la-li...
4,UEFA - Champions League,https://www.pinnacle.com/en/soccer/uefa-champi...
...,...,...
170,USA - NWSL Cup,https://www.pinnacle.com/en/soccer/usa-nwsl-cu...
171,USA - USL Championship,https://www.pinnacle.com/en/soccer/usa-usl-cha...
172,USA - USL League One,https://www.pinnacle.com/en/soccer/usa-usl-lea...
173,Venezuela - Primera Division,https://www.pinnacle.com/en/soccer/venezuela-p...


In [47]:
    #URL= 'https://www.pinnacle.com/en/soccer/england-premier-league/matchups'
def get_results(URL):
        import time 
        chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        browser = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
        browser.get(URL)
        time.sleep(10)
        htmlSource = browser.page_source
        soup= BeautifulSoup(browser.page_source, 'lxml')
        alle= pd.DataFrame()
        fest= pd.DataFrame()
        games= BeautifulSoup(browser.page_source, 'lxml').find_all('div',{'class':'style_row__3_aBC'})
        #dates= soup.find_all('div', {'class':'style_dateBar__2PS4O'})
        try: 
            for i,thing in enumerate(games): 
                teams= thing.find_all('span',{'class':'style_participantName__CNiJz ellipsis'})
                time= thing.find('span',{'class':'style_time__1_zpO ellipsis'})
                odds= thing.find_all('span',{'class':'price'})
                fest= pd.DataFrame({'time':time.text,
                         'home_team': teams[0].text,
                         'away_team': teams[1].text,
                         'PinnacleHome': odds[0].text,
                         'PinnacleDraw': odds[1].text,
                         'PinnacleAway': odds[2].text}, index= pd.Series(0))
                #alles['date']= dates[i]
                alle= alle.append(fest)
        except:
                alle= alle
        return(alle)

In [4]:
dreams= get_results('https://www.pinnacle.com/en/soccer/germany-bundesliga/matchups')

NameError: name 'get_results' is not defined

Now I've got to 
1. Fix the dates 
2. Add the scraping time for pinnacle odds 
3. Make sure there is not a big time-lag--> Time lag must be a parameter to be optimised 
4. Prepare the data for the merge 
5. 

In [262]:
import time 
chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
chrome_options = Options()
chrome_options.add_argument('--headless')
browser = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
browser.get('https://www.pinnacle.com/en/soccer/england-premier-league/matchups')
time.sleep(10)
htmlSource = browser.page_source
soup= BeautifulSoup(browser.page_source, 'lxml')

In [320]:
#ff= soup.find('div', {'class':'style_row__3_aBC style_lastRow__3h8Pm'})
#ff.find('span')
da= soup.find_all('div',{'class':'style_dateBar__2PS4O'})[3].text.split(',',1)[1][1:].replace(',','')
da
#da

'May 09 2021'

In [339]:
import dateutil.parser
pd.to_datetime(dateutil.parser.parse('May 09 2021'),format= '%d-%m-%Y')

Timestamp('2021-05-09 00:00:00')

In [330]:
datetime.strptime('May 09 2021','MMM' 'Do' 'Y')

ValueError: time data 'May 9 2021' does not match format 'MMMDoY'

### Fix this ASAP 

In [382]:
#def loop_Until_date:
#first= soup.find_all('div', {'class':'style_dateBar__2PS4O'})[0] 
#nextd= first.find_next('div', {'class':'style_dateBar__2PS4O'})

def get_results(URL):
    import time
    chrome_path= r'C:\Users\Iris\Documents\Nico\NicoUni\Practicum\Python\Projects\WebDrivers\chromedriver.exe'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
    browser.get(URL)
    time.sleep(10)
    htmlSource = browser.page_source
    soup= BeautifulSoup(browser.page_source, 'lxml')
    df= pd.DataFrame()
    error= []
    for element in soup.find_all('div',{'class':'style_dateBar__2PS4O'} ):
        if element.text ==  'Today':
            date= pd.to_datetime(datetime.now(), format="%d/%m/%Y")
        #if element.text == 'Tomorrow':
           # days_add= timedelta(days= 1)
           # date= pd.to_datetime(datetime.now(),format="%d/%m/%Y")+days_add
        else:
            date= element.text#.split(',',1)[1][1:].replace(',','')
        first= element.find_next('div',{'class':True})
        try:
            while first['class'] !=element['class']:
                if first['class']== ['style_row__3_aBC','style_lastRow__3h8Pm'] or first['class']== ['style_row__3_aBC']:
                    teams= first.find_all('span', {'class':'style_participantName__CNiJz ellipsis'})
                    time= first.find('span',{'class':'style_time__1_zpO ellipsis'})
                    odds= first.find_all('span',{'class':'price'})
                    fest= pd.DataFrame({'date':date,
                                    'time':time.text,
                         'home_team': teams[0].text,
                         'away_team': teams[1].text,
                         'PinnacleHome': odds[0].text,
                         'PinnacleDraw': odds[1].text,
                         'PinnacleAway': odds[2].text}, index= pd.Series(0))
                    df= df.append(fest)
                    first= first.find_next('div',{'class':True})
                else:
                    first= first.find_next('div',{'class':True})
        except ValueError:
            error.append(1)
        return(df)
           

In [387]:
df

,date,time,home_team,away_team,PinnacleHome,PinnacleDraw,PinnacleAway
0,2021-05-03 00:30:09.400650,16:00,Manchester United,Liverpool,2.960,3.480,2.520
0,2021-05-03 00:30:09.400650,19:00,West Brom,Wolves,2.870,3.270,2.730
0,2021-05-03 00:30:09.400650,21:15,Burnley,West Ham United,3.090,3.490,2.430
0,2021-05-07 00:00:00.000000,21:00,Leicester City,Newcastle United,1.462,4.880,7.310
0,2021-05-08 00:00:00.000000,13:30,Leeds United,Tottenham Hotspur,3.180,3.860,2.230
0,2021-05-08 00:00:00.000000,16:00,Sheffield United,Crystal Palace,2.890,3.340,2.660
0,2021-05-08 00:00:00.000000,18:30,Manchester City,Chelsea,1.793,3.730,4.880
0,2021-05-08 00:00:00.000000,21:15,Liverpool,Southampton,1.341,5.690,9.580
0,2021-05-09 00:00:00.000000,20:00,Arsenal,West Brom,1.460,4.690,7.430
0,2021-05-09 00:00:00.000000,15:05,Aston Villa,Manchester United,4.650,3.980,1.775


In [386]:
soup= BeautifulSoup(browser.page_source, 'lxml')
df= pd.DataFrame()
error= []
for element in soup.find_all('div',{'class':'style_dateBar__2PS4O'} ):
    print(element.text)
    if element.text == 'Today':
        date= pd.to_datetime(datetime.now(), format="%d/%m/%Y")
    elif element.text == 'Tomorrow':
        days_add= timedelta(days= 1)
        date= pd.to_datetime(datetime.now(),format="%d/%m/%Y")+days_add
    else:
        date= pd.to_datetime(dateutil.parser.parse(element.text.split(',',1)[1][1:].replace(',','')),format= '%d-%m-%Y')
    try:
        first= element.find_next('div',{'class':True})
        if len(element['class'])>0:
            for thin in element.find_all('div',{'class'}):
                if first['class']== ['style_row__3_aBC','style_lastRow__3h8Pm'] or first['class']== ['style_row__3_aBC']:
                    teams= first.find_all('span', {'class':'style_participantName__CNiJz ellipsis'})
                    time= first.find('span',{'class':'style_time__1_zpO ellipsis'})
                    odds= first.find_all('span',{'class':'price'})
                    fest= pd.DataFrame({'date':date,
                                        'time':time.text,
                         'home_team': teams[0].text,
                         'away_team': teams[1].text,
                         'PinnacleHome': odds[0].text,
                         'PinnacleDraw': odds[1].text,
                         'PinnacleAway': odds[2].text}, index= pd.Series(0))
                    df= df.append(fest)
                    first= first.find_next('div',{'class':True})
                else:
                    first= first.find_next('div',{'class':True})
        else:
            continue
    except ValueError:
        error.append(1)


Today
Fri, May 07, 2021
Sat, May 08, 2021
Sun, May 09, 2021
Mon, May 10, 2021
Tue, May 11, 2021
Wed, May 12, 2021
Thu, May 13, 2021


TypeError: 'NoneType' object is not subscriptable

In [48]:
# Very Important script
# links= get_pinn_leagues('https://www.pinnacle.com/en/soccer/leagues')

results= pd.DataFrame()
errors= []

for i,element in enumerate(links['link'][0:1]):
    try:
        df= get_results(element)
        df['league']= links['league'][i]
        results= results.append(df)
        
    except:
        errors.append(element) 